In [1]:
spark.conf.set(
  "fs.azure.account.key.xxxxx.blob.core.windows.net",
  "key")

In [2]:
dbutils.fs.ls("wasbs://signalmedia@xxxxx.blob.core.windows.net/")

In [3]:
 jsdf = spark.read.json("wasbs://signalmedia@xxxxxx.blob.core.windows.net/signalmedia-1m.jsonl")

In [4]:
 jsdf.take(1)

In [5]:
  jsdf.printSchema

In [6]:
dftitledesc = jsdf.select( "content", "title")

In [7]:
 dftitledesc.take(1)
 

In [8]:
from pyspark.sql.functions import concat
dff = dftitledesc.withColumn('textdata',concat('content', 'title'))

In [9]:
dffdata = dff.select('textdata')

In [10]:
dffdata.take(1)

In [11]:
from pyspark.sql.functions import split
dftokens = dffdata.withColumn('tokens',split('textdata',' '))

In [12]:
dftokens.printSchema()

In [13]:
dftokens.take(1)

In [14]:
from pyspark.ml.feature import Word2Vec

word2vec = Word2Vec(vectorSize=100,seed=42, inputCol="tokens", outputCol="model")
model = word2vec.fit(dftokens)



In [15]:
from pyspark.sql.functions import format_number as fmt
model.findSynonyms("King", 5).select("word", fmt("similarity", 5).alias("similarity")).show()

In [16]:
model.findSynonyms("Queen", 5).select("word", fmt("similarity", 5).alias("similarity")).show()

In [17]:
w2v_vector_map = model.getVectors()

In [18]:
king = w2v_vector_map.filter(w2v_vector_map["word"] == "King").select("vector").head()[0]
man = w2v_vector_map.filter(w2v_vector_map["word"] == "Man").select("vector").head()[0]
woman = w2v_vector_map.filter(w2v_vector_map["word"] == "Woman").select("vector").head()[0]
queen = (king - man) + woman

In [19]:
model.findSynonyms(queen, 10).select("word", fmt("similarity", 5).alias("similarity")).show()

In [22]:
from pyspark.ml.linalg import DenseVector
kv = DenseVector(king.head().toArray)
kv